In [9]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)
        
#RenderJSON(#json)

In [3]:
yelp_client_id = "BagZJOrnqKddLICuqFQeXw"
yelp_api = "tfUiSDCSapNaY3Bdg03l-aqMgFN4PpeW-TSLY-vhbV5mE_buz8pR6IUfndLtjhiNkIFl_XajJpAEDTJX4Vj8GW2FvSG1ZvKkZ-irjVNzajs57okH_1eKbhNQy4oAYHYx"


fs_id = "LI2VHLBCPFJEO1D1O5EMPDXA233B5QP4FGH442ERLK4INCNQ"
fs_s = "BAFOMI5B3PTIJFL5FVJIB0I4ZBQGXYXFPCAUQWHWT5EI3GWA"

In [4]:
import requests
import pandas as pd
import numpy as np
from IPython.display import JSON
import json
from pandas import json_normalize  

### YELP

In [6]:
term = 'coffee'
location = 'Calgary,AB'
radius = 1000
limit = 50



base_url = 'https://api.yelp.com/v3/businesses/search'

headers = {
         'Authorization': f'Bearer {yelp_api}'
}

params = {
    'term' : term,
    'location' : location,
    'radius': radius,
    'limit' : limit
}

res = requests.get(url=base_url, headers=headers,params=params)
result = res.json()
RenderJSON(result)

In [7]:
result['businesses'][0]['coordinates']

{'latitude': 51.0379649, 'longitude': -114.0787066}

In [10]:
yelpcalg = []
for i in result['businesses']:
    busdict = {}
    busdict['name'] = i['name']
    busdict['review_count'] = i['review_count']
    busdict['rating'] = i['rating']
    busdict['address'] = i['location']['address1']
    busdict['city'] = i['location']['city']
    busdict['zip_code'] = i['location']['zip_code']
    busdict['latitude'] = i['coordinates']['latitude']
    busdict['longitude'] = i['coordinates']['longitude']
    yelpcalg.append(busdict)

In [13]:
df_ycalgcoffee = pd.DataFrame(yelpcalg)
#df_ycalgcoffee.to_csv('yelpcalgcoffee.csv')

In [65]:
df_ycalgcoffee.head(5)

,name,review_count,rating,address,city,zip_code,latitude,longitude
0,Analog Coffee,151,4.0,740 17th Avenue SW,Calgary,T2S 0B7,51.037965,-114.078707
1,Milano Coffee Calgary,2,5.0,602 11 Avenue SW,Calgary,T2R 1J8,51.042915,-114.074824
2,Philosafy Coffee,52,4.5,632 17 Avenue SW,Calgary,T2S 0B4,51.037964,-114.076274
3,Phil & Sebastian Coffee Roasters,25,4.0,102 8 Avenue SW,Calgary,T2P 1B3,51.045679,-114.063508
4,il Caffe,2,5.0,207 9 Avenue SW,Calgary,T2P 1K3,51.044521,-114.066521


In [15]:
term = 'coffee'
location2 = 'Edmonton,AB'
radius = 1000
limit = 50



base_url = 'https://api.yelp.com/v3/businesses/search'

headers = {
         'Authorization': f'Bearer {yelp_api}'
}

params2 = {
    'term' : term,
    'location' : location2,
    'radius': radius,
    'limit' : limit
}

res2 = requests.get(url=base_url, headers=headers,params=params2)
result2 = res2.json()
RenderJSON(result2)

In [16]:
yelpedm = []
for j in result2['businesses']:
    busdicte = {}
    busdicte['name'] = j['name']
    busdicte['review_count'] = j['review_count']
    busdicte['rating'] = j['rating']
    busdicte['address'] = j['location']['address1']
    busdicte['city'] = j['location']['city']
    busdicte['zip_code'] = j['location']['zip_code']
    busdicte['latitude'] = j['coordinates']['latitude']
    busdicte['longitude'] = j['coordinates']['longitude']
    yelpedm.append(busdicte)

In [17]:
df_yedmcoffee = pd.DataFrame(yelpedm)
#df_edmcoffee = to_csv('yelpedmcoffee.csv')

## SQLALCH

In [18]:
import sqlite3 as sqlite
from sqlalchemy import create_engine

In [22]:
con = create_engine('sqlite:///coffee.db')

In [23]:
df_ycalgcoffee.to_sql('y_calg_coffee',con)
df_yedmcoffee.to_sql('y_edm_coffee',con)

## FOURSQUARE

In [25]:
url = "https://api.foursquare.com/v2/venues/explore?"
categoryId = "4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735" #coffee shops + cafes
limit = "50"
offset = "50"
v = "20201231"

near = "Calgary,AB"
radius = "1000" #area of calgary = ~820km^2

param = f'near={near}&categoryId={categoryId}&radius={radius}&limit={limit}&sortByPopularity=1&v={v}&client_id={fs_id}&client_secret={fs_s}'
res = requests.get(url+param)
fs_c = res.json()

In [28]:
RenderJSON(fs_c)

In [47]:
fs_c.update(fs_c.pop("response"))
fs_cd = json_normalize(fs_c['groups'],record_path='items')
fs_cd

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.postalCode,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,flags.outsideRadius,venue.location.neighborhood
0,e-0-4b1eb46ef964a520281e24e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b1eb46ef964a520281e24e3,Tim Hortons,"665 8th St Sw, Suite 120",51.047352,-114.081350,"[{'label': 'display', 'lat': 51.0473521, 'lng'...",T2P 3K7,...,Calgary,AB,Canada,"[665 8th St Sw, Suite 120, Calgary AB T2P 3K7,...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN,NaN
1,e-0-4b8c2b39f964a520f0c132e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b8c2b39f964a520f0c132e3,Tim Hortons,1211 12th Ave Sw,51.041855,-114.089450,"[{'label': 'display', 'lat': 51.0418549, 'lng'...",T3C 3S7,...,Calgary,AB,Canada,"[1211 12th Ave Sw (at 11 St.), Calgary AB T3C ...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],at 11 St.,NaN,NaN
2,e-0-4ca5e0eba6e08cfa9de07494-2,0,"[{'summary': 'This spot is popular', 'type': '...",4ca5e0eba6e08cfa9de07494,Tim Hortons,"520 3 Avenue Sw, Unit#205",51.050894,-114.072166,"[{'label': 'display', 'lat': 51.05089445321286...",T2P 0R3,...,Calgary,AB,Canada,"[520 3 Avenue Sw, Unit#205, Calgary AB T2P 0R3...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN,NaN
3,e-0-4b6c4525f964a520dc2c2ce3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4b6c4525f964a520dc2c2ce3,Tim Hortons,"444 5th Ave Sw, Unit 101",51.048789,-114.071276,"[{'label': 'display', 'lat': 51.04878928882179...",T2P 2T8,...,Calgary,AB,Canada,"[444 5th Ave Sw, Unit 101, Calgary AB T2P 2T8,...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN,NaN
4,e-0-4b3127c6f964a520570125e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b3127c6f964a520570125e3,Starbucks,550 - 6 Avenue SW,51.048033,-114.073237,"[{'label': 'display', 'lat': 51.048033, 'lng':...",T2P 0S2,...,Calgary,AB,Canada,"[550 - 6 Avenue SW (at 5 St. SW), Calgary AB T...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],at 5 St. SW,NaN,NaN
5,e-0-52ced5b4498e19cecabc8b45-5,0,"[{'summary': 'This spot is popular', 'type': '...",52ced5b4498e19cecabc8b45,Tim Hortons,640 5th Ave Sw,51.048990,-114.075663,"[{'label': 'display', 'lat': 51.04899022644555...",T2P 3G4,...,Calgary,AB,Canada,"[640 5th Ave Sw, Calgary AB T2P 3G4, Canada]","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN,NaN
6,e-0-4b0b1480f964a520902c23e3-6,0,"[{'summary': 'This spot is popular', 'type': '...",4b0b1480f964a520902c23e3,Kawa Espresso Bar,"#101, 1333 8th St. SW",51.040243,-114.081573,"[{'label': 'display', 'lat': 51.04024300120809...",T2R 1M6,...,Calgary,AB,Canada,"[#101, 1333 8th St. SW (S. of 12th Ave.), Calg...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],S. of 12th Ave.,True,NaN
7,e-0-55b2c1fc498e1bf8cf78682d-7,0,"[{'summary': 'This spot is popular', 'type': '...",55b2c1fc498e1bf8cf78682d,Vendome Cafe,940 2 Avenue NW,51.055138,-114.083323,"[{'label': 'display', 'lat': 51.05513753948826...",T2N 0E6,...,Calgary,AB,Canada,"[940 2 Avenue NW, Calgary AB T2N 0E6, Canada]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,NaN,NaN
8,e-0-4b09a65cf964a520061b23e3-8,0,"[{'summary': 'This spot is popular', 'type': '...",4b09a65cf964a520061b23e3,The Roasterie,314 10 St. NW,51.054396,-114.085905,"[{'label': 'display', 'lat': 51.05439555210016...",NaN,...,Calgary,AB,Canada,"[314 10 St. NW, Calgary AB, Canada]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,NaN,NaN
9,e-0-5aef866f95d98600390fd6f1-9,0,"[{'summary': 'This spot is popular', 'type': '...",5aef866f95d98600390fd6f1,Espresso Bar,NaN,51.053774,-114.086146,"[{'label': 'display', 'lat': 51.053774, 'lng':...",T2N 1V5,...,Calgary,AB,Canada,"[Calgary AB T2N 1V5, Can

In [55]:
cols = ["venue.name","venue.location.address","venue.location.postalCode","venue.location.lat","venue.location.lng"]

df_fcalgcoffee = fs_cd[cols]
df_fcalgcoffee = df_fcalgcoffee.rename(columns={
        "venue.name": "name",
        "venue.location.address": "address",
        "venue.location.postalCode": "zip_code",
        "venue.location.lat": "latitude",
        "venue.location.lng": "longitude",
})
df_fcalgcoffee.head()

,name,address,zip_code,latitude,longitude
0,Tim Hortons,"665 8th St Sw, Suite 120",T2P 3K7,51.047352,-114.081350
1,Tim Hortons,1211 12th Ave Sw,T3C 3S7,51.041855,-114.089450
2,Tim Hortons,"520 3 Avenue Sw, Unit#205",T2P 0R3,51.050894,-114.072166
3,Tim Hortons,"444 5th Ave Sw, Unit 101",T2P 2T8,51.048789,-114.071276
4,Starbucks,550 - 6 Avenue SW,T2P 0S2,51.048033,-114.073237


In [43]:
# fscalg = []
# for i in fs_c['response']['groups']:
#     fscdict = {}
#     fscdict['name'] = i['items']['venue']['name']
#     fscdict['rating'] = i['items']['venue']['rating']
#     fscdict['address'] = i['items']['venue']['location']['address']
#     fscdict['city'] = i['items']['venue']['location']['city']
#     fscdict['zip_code'] = i['items']['venue']['location']['postalCode']
#     fscdict['latitude'] = i['items']['venue']['location']['lat']
#     fscdict['longitude'] = i['items']['venue']['location']['lng']
#     fscalg.append(fscdict)

In [56]:
near2 = "Edmonton,AB" 
radius2 = "1000" #area of edmonton = ~680km^2

param2 = f'near={near2}&categoryId={categoryId}&radius={radius2}&limit={limit}&v={v}&client_id={fs_id}&client_secret={fs_s}'
res2 = requests.get(url+param2)
fs_e = res2.json()

In [59]:
fs_e.update(fs_e.pop('response'))

In [60]:
fs_ed = json_normalize(fs_e['groups'],record_path='items')
fs_ed

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.cc,...,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.city,venue.location.state,venue.location.postalCode,venue.location.crossStreet,venue.venuePage.id,venue.location.neighborhood
0,e-0-4cbf31ff19ceb1f7ee0822cd-0,0,"[{'summary': 'This spot is popular', 'type': '...",4cbf31ff19ceb1f7ee0822cd,Spinelli's Bar Italia,10878 95 St NW,53.555909,-113.485073,"[{'label': 'display', 'lat': 53.55590879567327...",CA,...,"[10878 95 St NW, Canada]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,NaN,NaN,NaN,NaN,NaN
1,e-0-55245475498e30f72b70ea7d-1,0,"[{'summary': 'This spot is popular', 'type': '...",55245475498e30f72b70ea7d,Little Brick,NaN,53.542125,-113.469391,"[{'label': 'display', 'lat': 53.542125, 'lng':...",CA,...,[Canada],"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN,NaN,NaN,NaN,NaN
2,e-0-4d4c4f3a855b54812579e68f-2,0,"[{'summary': 'This spot is popular', 'type': '...",4d4c4f3a855b54812579e68f,Sequel Café,10011 102 Ave NW,53.543281,-113.491101,"[{'label': 'display', 'lat': 53.54328056102945...",CA,...,"[10011 102 Ave NW, Edmonton AB, Canada]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],Edmonton,AB,NaN,NaN,NaN,NaN
3,e-0-4f347d4de4b0726329029d3c-3,0,"[{'summary': 'This spot is popular', 'type': '...",4f347d4de4b0726329029d3c,Remedy Cafe,10279 Jasper Ave. NW,53.540783,-113.497405,"[{'label': 'display', 'lat': 53.54078292362811...",CA,...,"[10279 Jasper Ave. NW, Edmonton AB T5J 2W4, Ca...","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],Edmonton,AB,T5J 2W4,NaN,NaN,NaN
4,e-0-533901fe498ef9b7a5d96ead-4,0,"[{'summary': 'This spot is popular', 'type': '...",533901fe498ef9b7a5d96ead,Second Cup Coffee Co.,7477 - 101st Avenue NW,53.540600,-113.442516,"[{'label': 'display', 'lat': 53.54060047346621...",CA,...,"[7477 - 101st Avenue NW (75 ST), Edmonton AB T...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],Edmonton,AB,T6A 3Z5,75 ST,NaN,NaN
5,e-0-4b05ca83f964a52060e322e3-5,0,"[{'summary': 'This spot is popular', 'type': '...",4b05ca83f964a52060e322e3,Credo Coffee,10134 104 St.,53.541738,-113.498980,"[{'label': 'display', 'lat': 53.54173773136662...",CA,...,"[10134 104 St., Edmonton AB T5J 1A7, Canada]","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],Edmonton,AB,T5J 1A7,NaN,NaN,NaN
6,e-0-5446c8ef498e07b4e52c7d77-6,0,"[{'summary': 'This spot is popular', 'type': '...",5446c8ef498e07b4e52c7d77,Transcend Coffee,10359 104 Street NW,53.546007,-113.499055,"[{'label': 'display', 'lat': 53.54600689066019...",CA,...,"[10359 104 Street NW, Edmonton AB, Canada]","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],Edmonton,AB,NaN,NaN,NaN,NaN
7,e-0-4b1535c7f964a5205da923e3-7,0,"[{'summary': 'This spot is popular', 'type': '...",4b1535c7f964a5205da923e3,Second Cup,#33 - 10025 Jasper Avenue,53.540654,-113.491221,"[{'label': 'display', 'lat': 53.54065358667587...",CA,...,"[#33 - 10025 Jasper Avenue (in Telus Plaza), E...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],Edmonton,AB,T5J 2B8,in Telus Plaza,NaN,NaN
8,e-0-4c229b3b99282d7f986d67b0-8,0,"[{'summary': 'This spot is popular', 'type': '...",4c229b3b99282d7f986d67b0,The Carrot Community Arts Coffeehouse,9351 118 Ave NW,53.570276,-113.485559,"[{'label': 'display', 'lat': 53.57027552598315...",CA,...,"[9351 118 Ave NW, Edmonton AB T5G 0N3, Canada]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],Edmonton,AB,T5G 0N3,NaN,41957205,NaN
9,e-0-54f78169498ec71694f84628-9,0,"[{'summary': 'This spot is popular', 'type': '...",54f78169498ec71694f84628,Coffee Bureau,NaN,53.540882,-113.501473,"[{'label': 'display', 'lat': 53.54088202486885...",CA,...,[Canada],"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df_fedmcoffee = fs_ed[cols]
df_fedmcoffee = df_fedmcoffee.rename(columns={
        "venue.name": "name",
        "venue.location.address": "address",
        "venue.location.postalCode": "zip_code",
        "venue.location.lat": "latitude",
        "venue.location.lng": "longitude",
})
df_fedmcoffee.head()

,name,address,zip_code,latitude,longitude
0,Spinelli's Bar Italia,10878 95 St NW,NaN,53.555909,-113.485073
1,Little Brick,NaN,NaN,53.542125,-113.469391
2,Sequel Café,10011 102 Ave NW,NaN,53.543281,-113.491101
3,Remedy Cafe,10279 Jasper Ave. NW,T5J 2W4,53.540783,-113.497405
4,Second Cup Coffee Co.,7477 - 101st Avenue NW,T6A 3Z5,53.540600,-113.442516


In [62]:
df_fcalgcoffee.to_sql('f_calg_coffee', con)
df_fedmcoffee.to_sql('f_edm_coffee', con)

In [63]:
pd.read_sql_query('select * from sqlite_master', con)

,type,name,tbl_name,rootpage,sql
0,table,y_calg_coffee,y_calg_coffee,2,"CREATE TABLE y_calg_coffee (\n\t""index"" BIGINT..."
1,index,ix_y_calg_coffee_index,y_calg_coffee,3,CREATE INDEX ix_y_calg_coffee_index ON y_calg_...
2,table,y_edm_coffee,y_edm_coffee,4,"CREATE TABLE y_edm_coffee (\n\t""index"" BIGINT,..."
3,index,ix_y_edm_coffee_index,y_edm_coffee,5,CREATE INDEX ix_y_edm_coffee_index ON y_edm_co...
4,table,f_calg_coffee,f_calg_coffee,6,"CREATE TABLE f_calg_coffee (\n\t""index"" BIGINT..."
5,index,ix_f_calg_coffee_index,f_calg_coffee,7,CREATE INDEX ix_f_calg_coffee_index ON f_calg_...
6,table,f_edm_coffee,f_edm_coffee,8,"CREATE TABLE f_edm_coffee (\n\t""index"" BIGINT,..."
7,index,ix_f_edm_coffee_index,f_edm_coffee,9,CREATE INDEX ix_f_edm_coffee_index ON f_edm_co...


## COMPARING

In [64]:
#querying foursquare didn't return ratings

#yelp dfs
#   df_ycalgcoffee
#   df_yedmcoffee
#foursquare dfs
#   df_fcalgcoffee
#   df_edmcoffee

df_yc = df_ycalgcoffee
df_ye = df_yedmcoffee
df_fc = df_fcalgcoffee
df_fe = df_fedmcoffee

In [67]:
#top 10 from yelp; no ratings from fs unless PAY LMAO
df_yc[(df_yc['name']!="Starbucks")&(df_yc['name']!="Tim Hortons")&(df_yc['name']!="Second Cup")].sort_values(by=['rating'], ascending=False).head(10)

,name,review_count,rating,address,city,zip_code,latitude,longitude
22,Cafe Eve,2,5.0,,Calgary,T2P 1C2,51.045810,-114.068190
25,Deville Coffee,2,5.0,225 6 Avenue SW,Calgary,T2P 2X6,51.047250,-114.066730
1,Milano Coffee Calgary,2,5.0,602 11 Avenue SW,Calgary,T2R 1J8,51.042915,-114.074824
4,il Caffe,2,5.0,207 9 Avenue SW,Calgary,T2P 1K3,51.044521,-114.066521
15,Element Cafe,1,5.0,331 17th Avenue SW,Calgary,T2S 0A5,51.037695,-114.070192
41,Deville Coffee,1,5.0,685 Centre St SW,Calgary,T2G 1S5,51.046669,-114.063188
9,Cafe Alchemist,5,5.0,850-2 Street Southwest,Calgary,T2P 0R8,51.045008,-114.067753
30,A1 Bodega and Café,2,4.5,1213–1st Street SW,Calgary,T2R 0V3,51.040941,-114.066177
33,Oasis Juice and Nutrition Bar,19,4.5,112-1012 1 St SW,Calgary,T2R 1M2,51.042979,-114.065396
35,Cookies By George,8,4.5,Toronto Dominion Square,Calgary,T2P 2Y9,51.045978,-114.069498


In [68]:
df_ye[(df_ye['name']!="Starbucks")&(df_ye['name']!="Tim Hortons")&(df_ye['name']!="Second Cup")].sort_values(by=['rating'], ascending=False).head(10)

,name,review_count,rating,address,city,zip_code,latitude,longitude
1,Vancia Cafe,2,4.5,103-9707 110 Street NW,Edmonton,T5K 2L9,53.534310,-113.509770
2,Why Not? Cafe And Bar,13,4.5,8534 109 Street NW,Edmonton,T6G 1E5,53.521660,-113.512330
4,Farrow Sandwiches & Coffee,85,4.5,8422 109 Street,Edmonton,T6G 1E2,53.520638,-113.512339
0,Transcend Coffee - Garneau,84,4.0,8708-109 Street,Edmonton,T6G 1E9,53.523022,-113.512361
5,SugarBowl,419,4.0,10922 88 Avenue NW,Edmonton,T6G 0Z1,53.523931,-113.513113
10,Upper Crust Cafe,61,4.0,10909 86 Ave NW,Edmonton,T6G 0W8,53.521740,-113.512560
11,Highlevel Diner,161,4.0,10912 88 Avenue NW,Edmonton,T6G 0Z1,53.523990,-113.512700
12,Zuppa Cafe & Pizza,27,4.0,9843-110 Street,Edmonton,T5K 2P5,53.537000,-113.509950
14,Rosso Pizzeria,91,4.0,8738 109 Street NW,Edmonton,T6G 1E9,53.523553,-113.512505
3,LEVA,127,3.5,11053 86 Avenue NW,Edmonton,T6G 0X1,53.521710,-113.517720


In [94]:
df_fc[(df_fc['name']!="Starbucks")&(df_fc['name']!="Tim Hortons")&(df_fc['name']!="Second Cup")].head(10)

,name,address,zip_code,latitude,longitude
6,Kawa Espresso Bar,"#101, 1333 8th St. SW",T2R 1M6,51.040243,-114.081573
7,Vendome Cafe,940 2 Avenue NW,T2N 0E6,51.055138,-114.083323
8,The Roasterie,314 10 St. NW,NaN,51.054396,-114.085905
9,Espresso Bar,NaN,T2N 1V5,51.053774,-114.086146
10,Societe Coffee Lounge,1223 11 Ave SW,T3C,51.042955,-114.089883
11,hexagon,NaN,NaN,51.052701,-114.088642
12,Redhead Japa Cafe,NaN,NaN,51.043156,-114.076065
13,Waves Coffee House,731 8th Street S.W.,T2P 3E5,51.046243,-114.081439
14,DeVille Luxury Coffee and Pastries,600 6th Ave SW,NaN,51.047805,-114.076284
15,Good Earth Cafe,707 7 Ave. SW,NaN,51.046884,-114.076360


In [70]:
#comparing calgary
dfyc = df_yc[(df_yc['name']!='Starbucks')&(df_yc['name']!='Tim Hortons')&(df_yc['name']!='Second Cup')]
dffc = df_fc[(df_fc['name']!='Starbucks')&(df_fc['name']!='Tim Hortons')&(df_fc['name']!='Second Cup')]

In [91]:
dfyc['name'].count()

42

In [92]:
dffc['name'].count()

33

In [72]:
import matplotlib.colors as colors
import folium
from IPython.display import display, HTML

In [82]:
calg = (51.0447,-114.0719)
m = folium.Map(location=calg, zoom_start=14)
opacity = 1

#dfyc
for name, lat, long in zip(dfyc['name'], dfyc['latitude'], dfyc['longitude']):
    label = '{}'.format(dfyc)
    folium.CircleMarker(
        location=(lat, long),
        popup=label,
        radius=5,
        color='red'
    ).add_to(m)


#dffc
for name, lat, long in zip(dffc['name'], dffc['latitude'], dffc['longitude']):
    label = '{}'.format(dffc)
    folium.CircleMarker(
        location=(lat, long),
        popup=label,
        radius=5,
        color='green'
    ).add_to(m)

m

In [83]:
dfye = df_ye[(df_ye['name']!='Starbucks')&(df_ye['name']!='Tim Hortons')&(df_ye['name']!='Second Cup')]
dffe = df_fe[(df_fe['name']!='Starbucks')&(df_fe['name']!='Tim Hortons')&(df_fe['name']!='Second Cup')]

In [90]:
dfye['name'].count()

13

In [89]:
dffe['name'].count()

34

In [84]:
edm = (53.5461,-113.4938)
k = folium.Map(location=edm, zoom_start=14)
opacity = 1

#dfyc
for name, lat, long in zip(dfye['name'], dfye['latitude'], dfye['longitude']):
    folium.CircleMarker(
        location=(lat, long),
        popup=label,
        radius=5,
        color='red'
    ).add_to(k)


#dffc
for name, lat, long in zip(dffe['name'], dffe['latitude'], dffe['longitude']):
    folium.CircleMarker(
        location=(lat, long),
        popup=label,
        radius=5,
        color='green'
    ).add_to(k)

k